In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
    del data['shop_score_service']
    del data['shop_score_description']
    del data['shop_review_positive_rate']
    del data['shop_score_delivery']
    return data

In [2]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [3]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click',
       'user_gender_click', 'day_click'],
      dtype='object')

In [4]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum', 'user_click', 'shop_click', 'brand_click',
       'user_gender_click', 'item_click', 'day_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num','shop_score_delivery_num', 'hour_num']

In [5]:
dataa = dataa[tezheng]

In [6]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [7]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 41)
2018-09-24 23:59:47
(357066, 41)
(121021, 41)


In [8]:

print('test')
test_a = pd.read_csv('test1.csv',index_col=False)
data2a = pd.read_csv('test1_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [9]:
data2a = data2a[tezheng]

In [10]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [11]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
del train['context_id']
del val['context_id']
del test_a['context_id']
del train['item_brand_id']
del val['item_brand_id']
del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(42888, 40)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [12]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [13]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:11
2018-09-23 00:00:11
1 days 23:59:33
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(42888, 37)


In [14]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# del train['item_id']
# del test_a['item_id']
# del val['item_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'item_id_pvnum', 'occupation_click', 'category_1_pvnum', 'user_click',
       'shop_click', 'brand_click', 'user_gender_click', 'item_click',
       'day_click', 'shop_score_service_num', 'shop_score_description_num',
       'shop_review_positive_rate_num', 'shop_score_delivery_num', 'hour_num',
       'category_0', 'category_1', 'category_2', 'property_0', 'property_1',
       'property_2', 'halfhour', 'user_count', 'item_count'],
      dtype='object')
(520975, 594)


In [15]:
# LogisticRegression?

In [16]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [17]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [22]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=64,
                        max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683901
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674842
[3]	valid_0's binary_logloss: 0.665957
[4]	valid_0's binary_logloss: 0.657237
[5]	valid_0's binary_logloss: 0.648693
[6]	valid_0's binary_logloss: 0.640309
[7]	valid_0's binary_logloss: 0.632086
[8]	valid_0's binary_logloss: 0.624017
[9]	valid_0's binary_logloss: 0.616093
[10]	valid_0's binary_logloss: 0.60832
[11]	valid_0's binary_logloss: 0.60069
[12]	valid_0's binary_logloss: 0.593193
[13]	valid_0's binary_logloss: 0.585835
[14]	valid_0's binary_logloss: 0.578613
[15]	valid_0's binary_logloss: 0.571518
[16]	valid_0's binary_logloss: 0.564545
[17]	valid_0's binary_logloss: 0.557692
[18]	valid_0's binary_logloss: 0.550959
[19]	valid_0's binary_logloss: 0.544349
[20]	valid_0's binary_logloss: 0.537853
[21]	valid_0's binary_logloss: 0.53147
[22]	valid_0's binary_logloss: 0.525181
[23]	valid_0's binary_logloss: 0.51901
[24]	valid_0's binary_logloss:

[204]	valid_0's binary_logloss: 0.124968
[205]	valid_0's binary_logloss: 0.124425
[206]	valid_0's binary_logloss: 0.123885
[207]	valid_0's binary_logloss: 0.123352
[208]	valid_0's binary_logloss: 0.122829
[209]	valid_0's binary_logloss: 0.122313
[210]	valid_0's binary_logloss: 0.1218
[211]	valid_0's binary_logloss: 0.121294
[212]	valid_0's binary_logloss: 0.120798
[213]	valid_0's binary_logloss: 0.120303
[214]	valid_0's binary_logloss: 0.119814
[215]	valid_0's binary_logloss: 0.119332
[216]	valid_0's binary_logloss: 0.118857
[217]	valid_0's binary_logloss: 0.118386
[218]	valid_0's binary_logloss: 0.117919
[219]	valid_0's binary_logloss: 0.117462
[220]	valid_0's binary_logloss: 0.117007
[221]	valid_0's binary_logloss: 0.116559
[222]	valid_0's binary_logloss: 0.116115
[223]	valid_0's binary_logloss: 0.115679
[224]	valid_0's binary_logloss: 0.115251
[225]	valid_0's binary_logloss: 0.114827
[226]	valid_0's binary_logloss: 0.114405
[227]	valid_0's binary_logloss: 0.113992
[228]	valid_0's bi

[404]	valid_0's binary_logloss: 0.0846904
[405]	valid_0's binary_logloss: 0.0846419
[406]	valid_0's binary_logloss: 0.084593
[407]	valid_0's binary_logloss: 0.0845454
[408]	valid_0's binary_logloss: 0.0845016
[409]	valid_0's binary_logloss: 0.0844611
[410]	valid_0's binary_logloss: 0.0844193
[411]	valid_0's binary_logloss: 0.0843798
[412]	valid_0's binary_logloss: 0.0843395
[413]	valid_0's binary_logloss: 0.0843025
[414]	valid_0's binary_logloss: 0.0842606
[415]	valid_0's binary_logloss: 0.0842179
[416]	valid_0's binary_logloss: 0.0841804
[417]	valid_0's binary_logloss: 0.0841406
[418]	valid_0's binary_logloss: 0.0841026
[419]	valid_0's binary_logloss: 0.084063
[420]	valid_0's binary_logloss: 0.0840242
[421]	valid_0's binary_logloss: 0.0839858
[422]	valid_0's binary_logloss: 0.0839509
[423]	valid_0's binary_logloss: 0.0839176
[424]	valid_0's binary_logloss: 0.0838793
[425]	valid_0's binary_logloss: 0.0838419
[426]	valid_0's binary_logloss: 0.0838092
[427]	valid_0's binary_logloss: 0.08

[601]	valid_0's binary_logloss: 0.0810037
[602]	valid_0's binary_logloss: 0.0809979
[603]	valid_0's binary_logloss: 0.0809916
[604]	valid_0's binary_logloss: 0.0809866
[605]	valid_0's binary_logloss: 0.0809793
[606]	valid_0's binary_logloss: 0.080975
[607]	valid_0's binary_logloss: 0.0809667
[608]	valid_0's binary_logloss: 0.0809576
[609]	valid_0's binary_logloss: 0.0809507
[610]	valid_0's binary_logloss: 0.0809466
[611]	valid_0's binary_logloss: 0.0809395
[612]	valid_0's binary_logloss: 0.0809332
[613]	valid_0's binary_logloss: 0.0809286
[614]	valid_0's binary_logloss: 0.0809243
[615]	valid_0's binary_logloss: 0.080917
[616]	valid_0's binary_logloss: 0.080912
[617]	valid_0's binary_logloss: 0.0809041
[618]	valid_0's binary_logloss: 0.0809004
[619]	valid_0's binary_logloss: 0.080896
[620]	valid_0's binary_logloss: 0.0808904
[621]	valid_0's binary_logloss: 0.0808871
[622]	valid_0's binary_logloss: 0.0808815
[623]	valid_0's binary_logloss: 0.0808746
[624]	valid_0's binary_logloss: 0.0808

[797]	valid_0's binary_logloss: 0.0803444
[798]	valid_0's binary_logloss: 0.0803429
[799]	valid_0's binary_logloss: 0.0803422
[800]	valid_0's binary_logloss: 0.0803417
[801]	valid_0's binary_logloss: 0.0803411
[802]	valid_0's binary_logloss: 0.0803384
[803]	valid_0's binary_logloss: 0.0803378
[804]	valid_0's binary_logloss: 0.0803375
[805]	valid_0's binary_logloss: 0.0803354
[806]	valid_0's binary_logloss: 0.0803359
[807]	valid_0's binary_logloss: 0.0803344
[808]	valid_0's binary_logloss: 0.0803339
[809]	valid_0's binary_logloss: 0.0803324
[810]	valid_0's binary_logloss: 0.0803302
[811]	valid_0's binary_logloss: 0.0803275
[812]	valid_0's binary_logloss: 0.0803245
[813]	valid_0's binary_logloss: 0.0803221
[814]	valid_0's binary_logloss: 0.0803209
[815]	valid_0's binary_logloss: 0.0803199
[816]	valid_0's binary_logloss: 0.0803183
[817]	valid_0's binary_logloss: 0.0803178
[818]	valid_0's binary_logloss: 0.0803167
[819]	valid_0's binary_logloss: 0.0803156
[820]	valid_0's binary_logloss: 0.

[996]	valid_0's binary_logloss: 0.0801659
[997]	valid_0's binary_logloss: 0.0801666
[998]	valid_0's binary_logloss: 0.0801661
[999]	valid_0's binary_logloss: 0.0801657
[1000]	valid_0's binary_logloss: 0.0801649
[1001]	valid_0's binary_logloss: 0.0801627
[1002]	valid_0's binary_logloss: 0.0801628
[1003]	valid_0's binary_logloss: 0.0801638
[1004]	valid_0's binary_logloss: 0.0801638
[1005]	valid_0's binary_logloss: 0.0801642
[1006]	valid_0's binary_logloss: 0.0801645
[1007]	valid_0's binary_logloss: 0.0801662
[1008]	valid_0's binary_logloss: 0.0801666
[1009]	valid_0's binary_logloss: 0.0801663
[1010]	valid_0's binary_logloss: 0.0801646
[1011]	valid_0's binary_logloss: 0.0801653
[1012]	valid_0's binary_logloss: 0.0801653
[1013]	valid_0's binary_logloss: 0.0801657
[1014]	valid_0's binary_logloss: 0.0801654
[1015]	valid_0's binary_logloss: 0.0801658
[1016]	valid_0's binary_logloss: 0.0801649
[1017]	valid_0's binary_logloss: 0.0801645
[1018]	valid_0's binary_logloss: 0.0801651
[1019]	valid_0'

[1188]	valid_0's binary_logloss: 0.080099
[1189]	valid_0's binary_logloss: 0.0800979
[1190]	valid_0's binary_logloss: 0.0800966
[1191]	valid_0's binary_logloss: 0.080097
[1192]	valid_0's binary_logloss: 0.0800978
[1193]	valid_0's binary_logloss: 0.0800979
[1194]	valid_0's binary_logloss: 0.0800978
[1195]	valid_0's binary_logloss: 0.0800964
[1196]	valid_0's binary_logloss: 0.0800959
[1197]	valid_0's binary_logloss: 0.0800966
[1198]	valid_0's binary_logloss: 0.0800968
[1199]	valid_0's binary_logloss: 0.080097
[1200]	valid_0's binary_logloss: 0.0800978
[1201]	valid_0's binary_logloss: 0.0800993
[1202]	valid_0's binary_logloss: 0.0800982
[1203]	valid_0's binary_logloss: 0.0800987
[1204]	valid_0's binary_logloss: 0.0800986
[1205]	valid_0's binary_logloss: 0.0800989
[1206]	valid_0's binary_logloss: 0.0800993
[1207]	valid_0's binary_logloss: 0.0800998
[1208]	valid_0's binary_logloss: 0.0801
[1209]	valid_0's binary_logloss: 0.0801015
[1210]	valid_0's binary_logloss: 0.0801008
[1211]	valid_0's 

[1385]	valid_0's binary_logloss: 0.0801296
[1386]	valid_0's binary_logloss: 0.0801288
[1387]	valid_0's binary_logloss: 0.0801288
[1388]	valid_0's binary_logloss: 0.0801291
[1389]	valid_0's binary_logloss: 0.0801298
[1390]	valid_0's binary_logloss: 0.0801291
Early stopping, best iteration is:
[1240]	valid_0's binary_logloss: 0.0800917


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=8, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=64, objective='binary', random_state=None,
       reg_alpha=2, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [23]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0802392979662

Start predicting...
0.0796834733104
0.0800916969695


In [24]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [25]:
y_sub_1.shape

(42888,)

In [26]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180419.txt',sep=" ",index=False)